In [4]:
from pennylane import qchem
import pennylane as qml

from pennylane import numpy as np
from openfermion import MolecularData
from openfermion.transforms import get_fermion_operator
from openfermion.ops import QubitOperator
import math
from itertools import combinations, product

In [5]:
def binom(n, k):
    return math.factorial(n) // math.factorial(k) // math.factorial(n - k)    

def append_to_dic(key,value,dic):
    if key in dic.keys():
        dic[key] += value
    else:
        dic[key] = value
        
def get_pauli(a,b):
    a = int(a)
    b = int(b)
    if a==0 and b==0:
        return [0,3] #|0><0|=1/2(I+Z)
    elif a==0 and b==1:
        return [1,2] #|0><1|=1/2(X+iY)
    elif a==1 and b==0:
        return [1,-2] #|1><0|=1/2(X-iY)
    elif a==1 and b==1:
        return [0,-3] #|1><1|=1/2(I-Z)

def get_h(term,h):
    xb = np.binary_repr(term[0],width=noq)
    yb = np.binary_repr(term[1],width=noq)
    tensor_list =[]
    # [p0,p1,...] get Pauli gates pi for each qubit i. 
    #pi is a list of linear combination of all pauli gates on ith qubit.
    for i in range(len(xb)):
        tensor_list.append(get_pauli(xb[i],yb[i]))           
    # Expand the linear combination. Now each list in pterms is a operation on all qubits.
    pterms = list(product(*tensor_list))
    pterms = [tuple(g) for g in pterms]
    coef = [h/(2**noq)]*len(pterms)     
    for cnt1,j in enumerate(pterms):
        for cnt2,k in enumerate(j):
            if k==-3:
                temp = list(pterms[cnt1])
                temp[cnt2]=3
                pterms[cnt1] = tuple(temp)
                coef[cnt1] *= -1
            elif k==2:
                temp = list(pterms[cnt1])
                temp[cnt2]=2
                pterms[cnt1] = tuple(temp)
                coef[cnt1] *= 1j
            elif k==-2:
                temp = list(pterms[cnt1])
                temp[cnt2]=2
                pterms[cnt1] = tuple(temp)
                coef[cnt1] *= -1j               
    return pterms,coef

In [6]:
def FermiOptoQubitOp(fermionic_hamiltonian,num_orbitals, num_electrons):  
    
    
    def convert_data(fermionic_hamiltonian):
        keylist = fermionic_hamiltonian.terms.keys()
        addlist = []
        for k in keylist:
            if (len(k)>2):
                if (k[1][0] == k[2][0]):
                    if ((k[1], k[2]) in fermionic_hamiltonian.terms.keys()):
                        l = [k[1], k[2]]
                        l = tuple(l)
                        fermionic_hamiltonian.terms[l] = fermionic_hamiltonian.terms[l] + fermionic_hamiltonian.terms[k]
                    else:
                        addlist.append([(k[1], k[2]), fermionic_hamiltonian.terms[k]])

                fermionic_hamiltonian.terms[k] = -1*fermionic_hamiltonian.terms[k]
        for k in addlist:
            fermionic_hamiltonian.terms[k[0]] = k[1]
        return fermionic_hamiltonian

    def convert_to_list(d):
        coeff = []
        terms = []
        for k in d.keys():
            temp = []
            for t in k:
                temp.append(t[0])
            if len(temp)!=4:
                terms.append(temp)
                coeff.append(d[k])
            else:
                if (temp[0]!=temp[1]) and (temp[2]!=temp[3]):
                    terms.append(temp)
                    coeff.append(d[k])
        return terms,coeff
 
    def fer_configs(orbitals, electrons):
        # output = np.array(list(fer_cofigs(orbitals, electrons)))
        for positions in combinations(range(orbitals), electrons):
            p = [0] * orbitals
            for i in positions:
                p[i] = 1
            yield p
    
    def one_elec(term):
        a,b = term
        k  = [] # before
        kp = [] # after
        parity=[]
        if len(set([a,b])) == 2:
            for cnt,conf in enumerate(Fconfigs):
                if conf[a]==0 and conf[b]==1:
                    k.append(cnt)
                    conf_af = conf.copy()
                    conf_af[a],conf_af[b]=conf_af[b],conf_af[a]
                    kp.append(np.where(np.all(Fconfigs==conf_af,axis=1))[0][0]) # find index for corresponded kp
                    parity.append((np.count_nonzero(conf[a+1:b])%2)*(-2)+1)
            return k,kp,parity
        elif len(set([a,b])) == 1:
            for cnt,conf in enumerate(Fconfigs):
                if conf[a]==conf[b] and conf[a]==1:
                    k.append(cnt)
                    kp.append(cnt) # find index for corresponded kp
                    parity.append(1)
            return k,kp,parity   

    def two_elec(term):
        a,b,c,d = term
        k  = [] # before
        kp = [] # after
        parity=[]
        
        # if a,b,c,d are four unique indices
        if len(set([a,b,c,d])) == 4:
            for cnt,conf in enumerate(Fconfigs):
                if conf[a]== 0 and conf[b]==0 and conf[c]== 1 and conf[d]==1:
                    k.append(cnt)
                    conf_af = conf.copy()
                    conf_af[a],conf_af[b],conf_af[c],conf_af[d]=conf_af[c],conf_af[d],conf_af[a],conf_af[b]
                    kp.append(np.where(np.all(Fconfigs==conf_af,axis=1))[0][0]) # find index for corresponded kp
                    parity.append(((np.count_nonzero(conf[a+1:c])%2)*(-2)+1)*((np.count_nonzero(conf[b+1:d])%2)*(-2)+1))
            return k,kp,parity   
        elif len(set([a,b,c,d])) == 2:
            for cnt,conf in enumerate(Fconfigs):
                if conf[a]== conf[d] and conf[c]==conf[d] and conf[a]== 1:
                    k.append(cnt)
                    kp.append(cnt) # find index for corresponded kp
                    parity.append(1)
            return k,kp,parity   
    
    
    def X(t):
        return qml.PauliX(t)
    def Y(t):
        return qml.PauliY(t)
    def Z(t):
        return qml.PauliZ(t)
    
    def get_pauli(n,t):
        if n==1:
            return X(t)
        elif n==2:
            return Y(t)
        elif n==3:
            return Z(t)
        
    def toPauli(pauli):
        for e in range(len(pauli)):
            if pauli[e]!=0:
                h = get_pauli(pauli[e],e)
                break
        for f in range(e+1,len(pauli)):
            if pauli[f]!=0:
                h @= get_pauli(pauli[f],f)
        return h
    
    def tensor_pauli(k,kp,parity,coef):
        #print(k,kp)
        Q_temp = {}
        for i in range(len(k)):
            lpauli,lcoeff = get_h((kp[i],k[i]),parity[i]*coef)
            for j,p in enumerate(lpauli):
                if all(a == 0 for a in p):
                    append_to_dic((),lcoeff[j],Q_temp)
                else:
                    append_to_dic(tuple(p),lcoeff[j],Q_temp)
        return Q_temp
    
    Fconfigs = np.array(list(fer_configs(num_orbitals, num_electrons)))
    Qconfigs={}
    
    f = convert_data(fermionic_hamiltonian)
    terms, coeffs= convert_to_list(f.terms)
    
    QubitOp = {}
    for cnt, term in enumerate(terms):
        if len(term)==0:
            append_to_dic((),coeffs[cnt],QubitOp)
        elif len(term)==2:
            k,kp,parity = one_elec(term)
            Q_temp = tensor_pauli(k,kp,parity,coeffs[cnt])
            for key in Q_temp.keys():
                append_to_dic(key,Q_temp[key],QubitOp)
        elif len(term)==4:
            k,kp,parity = two_elec(term)
            Q_temp = tensor_pauli(k,kp,parity,coeffs[cnt])
            for key in Q_temp.keys():
                append_to_dic(key,Q_temp[key],QubitOp)
    QubitOp = {key:val for key, val in QubitOp.items() if val != 0.0}      

    vals = []
    obs = []
    
    for i,key in enumerate(QubitOp.keys()):
        if len(key)==0:
            obs.append(qml.Identity(0))
            vals.append(QubitOp[key])
        else:
            obs.append(toPauli(key))
            vals.append(QubitOp[key])

    hamiltonian = qml.Hamiltonian(np.array(vals), obs)       
    return hamiltonian

In [7]:
symbols = ["H", "H"]
x = np.array([0.986, 1.610,  1.855,0.002], dtype=np.float64) # in Bohr
coordinate = np.array([0.0, 0.0, 0.0,x[0], x[1], 0.0], dtype=np.float64) # in Bohr

hf_file  = qchem.meanfield(symbols, coordinate,charge=0,mult = 1)
molecule = MolecularData(filename=hf_file.strip())
core, active = qchem.active_space(molecule.n_electrons, molecule.n_orbitals)
terms_molecular_hamiltonian = molecule.get_molecular_hamiltonian(occupied_indices=core, active_indices=active)
fermionic_hamiltonian = get_fermion_operator(terms_molecular_hamiltonian)

In [33]:
symbols = ["H", "H", "H"]
x = np.array([0.986, 1.610,  1.855,0.002], dtype=np.float64) # in Bohr
coordinate = np.array([0.0, 0.0, 0.0,x[0], x[1], 0.0, x[2],x[3], 0.0], dtype=np.float64) # in Bohr

hf_file  = qchem.meanfield(symbols, coordinate,charge=0,mult = 2)
molecule = MolecularData(filename=hf_file.strip())
core, active = qchem.active_space(molecule.n_electrons, molecule.n_orbitals)
terms_molecular_hamiltonian = molecule.get_molecular_hamiltonian(occupied_indices=core, active_indices=active)
fermionic_hamiltonian = get_fermion_operator(terms_molecular_hamiltonian)

In [8]:
# input
num_electrons = molecule.n_electrons
num_orbitals = molecule.n_orbitals*2
noq = math.ceil(np.log2(binom(num_orbitals,num_electrons)))

In [9]:
H = FermiOptoQubitOp(fermionic_hamiltonian,num_orbitals, num_electrons)
H

<Hamiltonian: terms=16, wires=[0, 1, 2]>

In [10]:
print(H)

  ((-0.1339491298487199+0j)) [Z2]
+ ((-0.08357743213602559+0j)) [Z0]
+ ((-0.049182627042925406+0j)) [X2]
+ ((-0.003323113630281739+0j)) [Z1]
+ ((0.6270997250714615+0j)) [I0]
+ ((-0.18432082756141427+0j)) [Z0 Z1]
+ ((-0.13394912984871993+0j)) [Z1 Z2]
+ ((-0.049182627042925406+0j)) [Z0 X2]
+ (0.049182627042925406j) [X0 Y2]
+ (0.049182627042925406j) [Y0 X2]
+ ((0.044670442742874746+0j)) [Z0 Z2]
+ ((0.049182627042925406+0j)) [Z1 X2]
+ (0.049182627042925406j) [X0 Z1 Y2]
+ (0.049182627042925406j) [Y0 Z1 X2]
+ ((0.044670442742874746+0j)) [Z0 Z1 Z2]
+ ((0.049182627042925406+0j)) [Z0 Z1 X2]


In [12]:
h_qiskit = to_qiskit_hamiltonian(H,3)

ModuleNotFoundError: No module named 'qiskit'

In [11]:
from qiskit.aqua.operators import WeightedPauliOperator

ModuleNotFoundError: No module named 'qiskit'

In [10]:
def to_qiskit_hamiltonian(hamiltonian, noq=None):
    """
    
    Input: 
        Pennylane Hamiltonan: hamiltonian
        Number of qubits: noq
    
    Return: 
        QISKit Hamiltonian: qubit_op 
    
    """
    
    from pennylane.operation import Observable, Tensor
    from qiskit.aqua.operators import WeightedPauliOperator
    
    OBS_MAP = {"PauliX": "X", "PauliY": "Y", "PauliZ": "Z", "Hadamard": "H", "Identity": "I"}
    pauli_dict= {"paulis": []}
    
    if len(hamiltonian.coeffs) != len(hamiltonian.ops):
        raise ValueError("Could not create valid Hamiltonian; "
                "number of coefficients and operators does not match.")

    for obs in hamiltonian.ops:
        if not isinstance(obs, Observable):
            raise ValueError("Could not create circuits. Some or all observables are not valid.")             
                
    for i, obs in enumerate(hamiltonian.ops):
        if isinstance(obs, Tensor):
            pauli_term = ["I"] * noq
            for j in obs.obs:
                pauli_term[j.wires[0]] = OBS_MAP[j.name]
        
        elif isinstance(obs, Observable):
            pauli_term = ["I"] * noq
            pauli_term[obs.wires[0]] = OBS_MAP[obs.name]
            
        term = "".join(pauli_term)
        op = {"label": term}
        
        op["coeff"] = {"real": np.real(hamiltonian.coeffs[i]),"imag": np.imag(hamiltonian.coeffs[i])}
             
        pauli_dict["paulis"].append(op)
        
    qubit_op = WeightedPauliOperator.from_dict(pauli_dict)
    
    return qubit_op

NameError: name 'two_elec' is not defined